Importing libraries

In [1]:
from google.colab import drive
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd  # To save signal data to CSV
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler
from matplotlib.backends.backend_pdf import PdfPages  # For saving to PDF

In [2]:
!pip install wfdb
import wfdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 4.7 MB/s eta 0:00:00


Mounting Google Drive

In [3]:
# Mounting the Google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:

# Define the base directory for the dataset
base_dir = '/content/gdrive/MyDrive/Records/Database'

# List all patient folders
patient_folders = [f.path for f in os.scandir(base_dir) if f.is_dir()]

count = 0  # Initialize count globally

# Function to apply a Butterworth high-pass filter for baseline wandering removal
def remove_baseline_wandering(signal, sampling_rate, cutoff=0.5, order=5):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

# Function to plot and process ECG signals for a patient and save as separate PDFs and CSVs
def plot_ecg_signals(patient_folder, sampling_rate=1000):
    global count  # Declare count as global to modify the global variable
    files = os.listdir(patient_folder)

    for file in files:
        if file.endswith(".dat"):
            record_name = os.path.join(patient_folder, file.split(".")[0])

            # Load the record
            record = wfdb.rdrecord(record_name)
            signals = record.p_signal
            count += 1

            # Segment the data from 20,000 to 23,000 sample indexes (as an example)
            segment = signals[20000:23000, :]  # Adjust the indices as needed

            # Combine the segmented signals (optional: you can use only specific leads)
            combined_segment = np.mean(segment, axis=1)

            # Remove baseline wandering using a high-pass filter
            filtered_segment = remove_baseline_wandering(combined_segment, sampling_rate)

            # Normalize the signal
            filtered_segment_scaled = StandardScaler().fit_transform(filtered_segment.reshape(-1, 1)).flatten()

            # Define the filenames for PDF and CSV for each individual file
            pdf_filename = os.path.join(patient_folder, f'{file.split(".")[0]}_filtered_ecg_signals.pdf')
            csv_filename = os.path.join(patient_folder, f'{file.split(".")[0]}_signal_coordinates.csv')

            # Save the plot as a PDF
            with PdfPages(pdf_filename) as pdf:
                # Plot the processed signal (filtered and normalized)
                plt.figure(figsize=(20, 5))
                plt.plot(filtered_segment_scaled)
                plt.title(f'Processed ECG Signal from {file} (Filtered and Normalized)')
                plt.xlabel('Time (samples)')
                plt.ylabel('Amplitude (Normalized)')

                # Save the plot to the PDF
                pdf.savefig()  # Saves the current figure to the PDF
                plt.close()    # Close the plot to avoid displaying it multiple times

            # Save time indices and normalized amplitudes to CSV
            with open(csv_filename, 'w') as csv_file:
                csv_file.write('Time (samples),Amplitude (Normalized)\n')  # Write CSV header
                time_indices = np.arange(20000, 23000)  # Time indices for the segment
                for t, amp in zip(time_indices, filtered_segment_scaled):
                    csv_file.write(f'{t},{amp}\n')

            print(f"Processed {file} - {count} files so far")

# Iterate over each patient
for patient_folder in patient_folders:
    print(f'Processing {patient_folder}')
    plot_ecg_signals(patient_folder)

print(f"Total files processed: {count}")

Processing /content/gdrive/MyDrive/Records/Database/patient288
Processed s0549_re.dat - 1 files so far
Processing /content/gdrive/MyDrive/Records/Database/patient289
Processed s0550_re.dat - 2 files so far
Processing /content/gdrive/MyDrive/Records/Database/patient286
Processed s0546_re.dat - 3 files so far
Processing /content/gdrive/MyDrive/Records/Database/patient287
Processed s0548_re.dat - 4 files so far
Processed s0547_re.dat - 5 files so far
Processing /content/gdrive/MyDrive/Records/Database/patient292
Processed s0556_re.dat - 6 files so far
Processed s0555_re.dat - 7 files so far
Processing /content/gdrive/MyDrive/Records/Database/patient290
Processed s0553_re.dat - 8 files so far
Processing /content/gdrive/MyDrive/Records/Database/patient294
Processed s0559_re.dat - 9 files so far
Processing /content/gdrive/MyDrive/Records/Database/patient293
Processed s0557_re.dat - 10 files so far
Processed s0558_re.dat - 11 files so far
Processing /content/gdrive/MyDrive/Records/Database/pa

Extracting the features

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.fft import fft
from scipy.stats import entropy
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler

# Function to extract relevant features from ECG signal
def extract_ecg_features(time, signal, sampling_rate=1000):
    # Feature 1: Mean of the ECG signal
    mean_amplitude = np.mean(signal)

    # Feature 2: Standard deviation of the ECG signal
    std_amplitude = np.std(signal)

    # Feature 3: Variance of the ECG signal
    variance_amplitude = np.var(signal)

    # Feature 4: Frequency domain feature (dominant frequency)
    fft_values = fft(signal)
    fft_magnitude = np.abs(fft_values)
    freqs = np.fft.fftfreq(len(fft_values), 1 / sampling_rate)
    dominant_frequency = freqs[np.argmax(fft_magnitude[:len(freqs)//2])]  # Extract dominant frequency

    # Feature 5: Entropy-based measure
    signal_entropy = entropy(np.abs(signal))

    # Feature 6: Detect R peaks
    peaks, _ = find_peaks(signal, distance=150)  # Adjust distance based on sampling rate
    rr_intervals = np.diff(time[peaks])
    mean_rr_interval = np.mean(rr_intervals) if len(rr_intervals) > 0 else np.nan
    std_rr_interval = np.std(rr_intervals) if len(rr_intervals) > 0 else np.nan

    # Feature 7: QRS duration (approximated by the width of the peaks)
    qrs_duration = np.mean(np.diff(peaks)) if len(peaks) > 1 else np.nan

    # Feature 8: Number of QRS complexes (number of peaks)
    num_peaks = len(peaks)

    # Peak detection for P, Q, R, S, and T waves
    pr_interval = np.nan
    qt_interval = np.nan
    st_segment = np.nan
    t_wave_morphology = np.nan

    if len(peaks) > 1:
        for i in range(1, len(peaks)):
            r_peak = peaks[i]
            prev_r_peak = peaks[i-1]
            pr_interval = (r_peak - prev_r_peak) / sampling_rate  # Time difference between consecutive peaks

            qt_interval = (r_peak + 0.4 * sampling_rate) / sampling_rate  # Approximation
            st_segment = (r_peak + 0.2 * sampling_rate) / sampling_rate  # Approximation

            t_wave_morphology = np.max(signal[r_peak:r_peak + int(0.4 * sampling_rate)]) - np.min(signal[r_peak:r_peak + int(0.4 * sampling_rate)])

    # Store extracted features in a dictionary
    features = {
        'mean_amplitude': mean_amplitude,
        'std_amplitude': std_amplitude,
        'variance_amplitude': variance_amplitude,
        'dominant_frequency': dominant_frequency,
        'signal_entropy': signal_entropy,
        'mean_rr_interval': mean_rr_interval,
        'std_rr_interval': std_rr_interval,
        'qrs_duration': qrs_duration,
        'num_peaks': num_peaks,
        'pr_interval': pr_interval,
        'qt_interval': qt_interval,
        'st_segment': st_segment,
        't_wave_morphology': t_wave_morphology
    }

    return features

# Function to process ECG files, extract features, and append to a single DataFrame
def process_ecg_files(patient_folder):
    all_features = []

    # List all CSV files in the patient folder
    csv_files = [f for f in os.listdir(patient_folder) if f.endswith('_signal_coordinates.csv')]

    for csv_file in csv_files:
        csv_path = os.path.join(patient_folder, csv_file)

        # Load the ECG signal data
        df = pd.read_csv(csv_path)
        time = df['Time (samples)'].values
        amplitude = df['Amplitude (Normalized)'].values

        # Extract features from the ECG signal
        features = extract_ecg_features(time, amplitude)

        # Append file and patient folder information to the features
        features['patient_folder'] = os.path.basename(patient_folder)
        features['csv_file'] = csv_file

        # Add features to the list
        all_features.append(features)

    return all_features

# Define the base directory where the patient data is located
base_dir = '/content/gdrive/MyDrive/Records/Database'

# Initialize an empty list to store features from all patients
all_patient_features = []

# Process each patient folder
patient_folders = [f.path for f in os.scandir(base_dir) if f.is_dir()]
for patient_folder in patient_folders:
    patient_features = process_ecg_files(patient_folder)
    all_patient_features.extend(patient_features)  # Append all patient features to the main list
    print(f'Processed folder: {patient_folder}')

# Convert all features to a DataFrame and save to the specified CSV file
output_csv = '/content/gdrive/MyDrive/Records/extracted_features.csv'
features_df = pd.DataFrame(all_patient_features)
features_df.to_csv(output_csv, index=False)
print(f'All features saved to {output_csv}')


Processed folder: /content/gdrive/MyDrive/Records/Database/patient288
Processed folder: /content/gdrive/MyDrive/Records/Database/patient289
Processed folder: /content/gdrive/MyDrive/Records/Database/patient286
Processed folder: /content/gdrive/MyDrive/Records/Database/patient287
Processed folder: /content/gdrive/MyDrive/Records/Database/patient292
Processed folder: /content/gdrive/MyDrive/Records/Database/patient290
Processed folder: /content/gdrive/MyDrive/Records/Database/patient294
Processed folder: /content/gdrive/MyDrive/Records/Database/patient293
Processed folder: /content/gdrive/MyDrive/Records/Database/patient291
Processed folder: /content/gdrive/MyDrive/Records/Database/patient285
Processed folder: /content/gdrive/MyDrive/Records/Database/patient276
Processed folder: /content/gdrive/MyDrive/Records/Database/patient284
Processed folder: /content/gdrive/MyDrive/Records/Database/patient275
Processed folder: /content/gdrive/MyDrive/Records/Database/patient279
Processed folder: /c

To get the patient details

In [6]:
import pandas as pd

def read_header(hea_file):
    with open(hea_file, 'r') as f:
        header = f.readlines()

    # Extract relevant header lines
    header_info = {}
    for line in header:
        if line.startswith('#'):
            key_value = line[2:].strip().split(': ')
            if len(key_value) == 2:
                key, value = key_value
                header_info[key] = value

    # Convert to DataFrame
    return pd.DataFrame([header_info])

# List of .hea files to process
hea_files = [
   'patient001/s0010_re.hea',
'patient001/s0014lre.hea',
'patient001/s0016lre.hea',
'patient002/s0015lre.hea',
'patient003/s0017lre.hea',
'patient004/s0020are.hea',
'patient004/s0020bre.hea',
'patient005/s0021are.hea',
'patient005/s0021bre.hea',
'patient005/s0025lre.hea',
'patient005/s0031lre.hea',
'patient005/s0101lre.hea',
'patient006/s0022lre.hea',
'patient006/s0027lre.hea',
'patient006/s0064lre.hea',
'patient007/s0026lre.hea',
'patient007/s0029lre.hea',
'patient007/s0038lre.hea',
'patient007/s0078lre.hea',
'patient008/s0028lre.hea',
'patient008/s0037lre.hea',
'patient008/s0068lre.hea',
'patient009/s0035_re.hea',
'patient010/s0036lre.hea',
'patient010/s0042lre.hea',
'patient010/s0061lre.hea',
'patient011/s0039lre.hea',
'patient011/s0044lre.hea',
'patient011/s0049lre.hea',
'patient011/s0067lre.hea',
'patient012/s0043lre.hea',
'patient012/s0050lre.hea',
'patient013/s0045lre.hea',
'patient013/s0051lre.hea',
'patient013/s0072lre.hea',
'patient014/s0046lre.hea',
'patient014/s0056lre.hea',
'patient014/s0071lre.hea',
'patient015/s0047lre.hea',
'patient015/s0057lre.hea',
'patient015/s0152lre.hea',
'patient016/s0052lre.hea',
'patient016/s0060lre.hea',
'patient016/s0076lre.hea',
'patient017/s0053lre.hea',
'patient017/s0055lre.hea',
'patient017/s0063lre.hea',
'patient017/s0075lre.hea',
'patient018/s0054lre.hea',
'patient018/s0059lre.hea',
'patient018/s0082lre.hea',
'patient019/s0058lre.hea',
'patient019/s0070lre.hea',
'patient019/s0077lre.hea',
'patient020/s0062lre.hea',
'patient020/s0069lre.hea',
'patient020/s0079lre.hea',
'patient021/s0065lre.hea',
'patient021/s0073lre.hea',
'patient021/s0097lre.hea',
'patient022/s0066lre.hea',
'patient022/s0074lre.hea',
'patient022/s0149lre.hea',
'patient023/s0080lre.hea',
'patient023/s0081lre.hea',
'patient023/s0085lre.hea',
'patient023/s0103lre.hea',
'patient024/s0083lre.hea',
'patient024/s0084lre.hea',
'patient024/s0086lre.hea',
'patient024/s0094lre.hea',
'patient025/s0087lre.hea',
'patient025/s0091lre.hea',
'patient025/s0150lre.hea',
'patient026/s0088lre.hea',
'patient026/s0095lre.hea',
'patient027/s0089lre.hea',
'patient027/s0096lre.hea',
'patient027/s0151lre.hea',
'patient028/s0090lre.hea',
'patient028/s0093lre.hea',
'patient028/s0108lre.hea',
'patient029/s0092lre.hea',
'patient029/s0098lre.hea',
'patient029/s0122lre.hea',
'patient030/s0099lre.hea',
'patient030/s0107lre.hea',
'patient030/s0117lre.hea',
'patient030/s0153lre.hea',
'patient031/s0100lre.hea',
'patient031/s0104lre.hea',
'patient031/s0114lre.hea',
'patient031/s0127lre.hea',
'patient032/s0102lre.hea',
'patient032/s0106lre.hea',
'patient032/s0115lre.hea',
'patient032/s0165lre.hea',
'patient033/s0105lre.hea',
'patient033/s0113lre.hea',
'patient033/s0121lre.hea',
'patient033/s0157lre.hea',
'patient034/s0109lre.hea',
'patient034/s0118lre.hea',
'patient034/s0123lre.hea',
'patient034/s0158lre.hea',
'patient035/s0110lre.hea',
'patient035/s0119lre.hea',
'patient035/s0124lre.hea',
'patient035/s0145lre.hea',
'patient036/s0111lre.hea',
'patient036/s0116lre.hea',
'patient036/s0126lre.hea',
'patient037/s0112lre.hea',
'patient037/s0120lre.hea',
'patient038/s0125lre.hea',
'patient038/s0128lre.hea',
'patient038/s0162lre.hea',
'patient039/s0129lre.hea',
'patient039/s0134lre.hea',
'patient039/s0164lre.hea',
'patient040/s0130lre.hea',
'patient040/s0131lre.hea',
'patient040/s0133lre.hea',
'patient040/s0219lre.hea',
'patient041/s0132lre.hea',
'patient041/s0136lre.hea',
'patient041/s0138lre.hea',
'patient041/s0276lre.hea',
'patient042/s0135lre.hea',
'patient042/s0137lre.hea',
'patient042/s0140lre.hea',
'patient042/s0347lre.hea',
'patient043/s0141lre.hea',
'patient043/s0144lre.hea',
'patient043/s0278lre.hea',
'patient044/s0142lre.hea',
'patient044/s0143lre.hea',
'patient044/s0146lre.hea',
'patient044/s0159lre.hea',
'patient045/s0147lre.hea',
'patient045/s0148lre.hea',
'patient045/s0155lre.hea',
'patient045/s0217lre.hea',
'patient046/s0156lre.hea',
'patient046/s0161lre.hea',
'patient046/s0168lre.hea',
'patient046/s0184lre.hea',
'patient047/s0160lre.hea',
'patient047/s0163lre.hea',
'patient047/s0167lre.hea',
'patient048/s0171lre.hea',
'patient048/s0172lre.hea',
'patient048/s0180lre.hea',
'patient048/s0277lre.hea',
'patient049/s0173lre.hea',
'patient049/s0178lre.hea',
'patient049/s0186lre.hea',
'patient049/s0314lre.hea',
'patient050/s0174lre.hea',
'patient050/s0177lre.hea',
'patient050/s0185lre.hea',
'patient050/s0215lre.hea',
'patient051/s0179lre.hea',
'patient051/s0181lre.hea',
'patient051/s0187lre.hea',
'patient051/s0213lre.hea',
'patient052/s0190lre.hea',
'patient053/s0191lre.hea',
'patient054/s0192lre.hea',
'patient054/s0195lre.hea',
'patient054/s0197lre.hea',
'patient054/s0218lre.hea',
'patient055/s0194lre.hea',
'patient056/s0196lre.hea',
'patient057/s0198lre.hea',
'patient058/s0216lre.hea',
'patient059/s0208lre.hea',
'patient060/s0209lre.hea',
'patient061/s0210lre.hea',
'patient062/s0212lre.hea',
'patient063/s0214lre.hea',
'patient064/s0220lre.hea',
'patient065/s0221lre.hea',
'patient065/s0226lre.hea',
'patient065/s0229lre.hea',
'patient065/s0282lre.hea',
'patient066/s0225lre.hea',
'patient066/s0231lre.hea',
'patient066/s0280lre.hea',
'patient067/s0227lre.hea',
'patient067/s0230lre.hea',
'patient067/s0283lre.hea',
'patient068/s0228lre.hea',
'patient069/s0232lre.hea',
'patient069/s0233lre.hea',
'patient069/s0234lre.hea',
'patient069/s0284lre.hea',
'patient070/s0235lre.hea',
'patient071/s0236lre.hea',
'patient072/s0237lre.hea',
'patient072/s0240lre.hea',
'patient072/s0244lre.hea',
'patient072/s0318lre.hea',
'patient073/s0238lre.hea',
'patient073/s0243lre.hea',
'patient073/s0249lre.hea',
'patient073/s0252lre.hea',
'patient074/s0239lre.hea',
'patient074/s0241lre.hea',
'patient074/s0245lre.hea',
'patient074/s0406lre.hea',
'patient075/s0242lre.hea',
'patient075/s0246lre.hea',
'patient075/s0248lre.hea',
'patient075/s0327lre.hea',
'patient076/s0247lre.hea',
'patient076/s0250lre.hea',
'patient076/s0253lre.hea',
'patient076/s0319lre.hea',
'patient077/s0251lre.hea',
'patient077/s0254lre.hea',
'patient077/s0258lre.hea',
'patient077/s0285lre.hea',
'patient078/s0255lre.hea',
'patient078/s0259lre.hea',
'patient078/s0262lre.hea',
'patient078/s0317lre.hea',
'patient079/s0256lre.hea',
'patient079/s0257lre.hea',
'patient079/s0263lre.hea',
'patient079/s0269lre.hea',
'patient080/s0260lre.hea',
'patient080/s0261lre.hea',
'patient080/s0265lre.hea',
'patient080/s0315lre.hea',
'patient081/s0264lre.hea',
'patient081/s0266lre.hea',
'patient081/s0270lre.hea',
'patient081/s0346lre.hea',
'patient082/s0267lre.hea',
'patient082/s0271lre.hea',
'patient082/s0279lre.hea',
'patient082/s0320lre.hea',
'patient083/s0268lre.hea',
'patient083/s0272lre.hea',
'patient083/s0286lre.hea',
'patient083/s0290lre.hea',
'patient084/s0281lre.hea',
'patient084/s0288lre.hea',
'patient084/s0289lre.hea',
'patient084/s0313lre.hea',
'patient085/s0296lre.hea',
'patient085/s0297lre.hea',
'patient085/s0298lre.hea',
'patient085/s0345lre.hea',
'patient086/s0316lre.hea',
'patient087/s0321lre.hea',
'patient087/s0326lre.hea',
'patient087/s0330lre.hea',
'patient088/s0339lre.hea',
'patient088/s0343lre.hea',
'patient088/s0352lre.hea',
'patient088/s0413lre.hea',
'patient089/s0344lre.hea',
'patient089/s0355lre.hea',
'patient089/s0359lre.hea',
'patient089/s0372lre.hea',
'patient090/s0348lre.hea',
'patient090/s0356lre.hea',
'patient090/s0360lre.hea',
'patient090/s0418lre.hea',
'patient091/s0353lre.hea',
'patient091/s0357lre.hea',
'patient091/s0361lre.hea',
'patient091/s0408lre.hea',
'patient092/s0354lre.hea',
'patient092/s0358lre.hea',
'patient092/s0362lre.hea',
'patient092/s0411lre.hea',
'patient093/s0367lre.hea',
'patient093/s0371lre.hea',
'patient093/s0375lre.hea',
'patient093/s0378lre.hea',
'patient093/s0396lre.hea',
'patient094/s0368lre.hea',
'patient094/s0370lre.hea',
'patient094/s0376lre.hea',
'patient094/s0412lre.hea',
'patient095/s0369lre.hea',
'patient095/s0373lre.hea',
'patient095/s0377lre.hea',
'patient095/s0417lre.hea',
'patient096/s0379lre.hea',
'patient096/s0381lre.hea',
'patient096/s0385lre.hea',
'patient096/s0395lre.hea',
'patient097/s0380lre.hea',
'patient097/s0382lre.hea',
'patient097/s0384lre.hea',
'patient097/s0394lre.hea',
'patient098/s0386lre.hea',
'patient098/s0389lre.hea',
'patient098/s0398lre.hea',
'patient098/s0409lre.hea',
'patient099/s0387lre.hea',
'patient099/s0388lre.hea',
'patient099/s0397lre.hea',
'patient099/s0419lre.hea',
'patient100/s0399lre.hea',
'patient100/s0401lre.hea',
'patient100/s0407lre.hea',
'patient101/s0400lre.hea',
'patient101/s0410lre.hea',
'patient101/s0414lre.hea',
'patient102/s0416lre.hea',
'patient103/s0332lre.hea',
'patient104/s0306lre.hea',
'patient105/s0303lre.hea',
'patient106/s0030_re.hea',
'patient107/s0199_re.hea',
'patient108/s0013_re.hea',
'patient109/s0349lre.hea',
'patient110/s0003_re.hea',
'patient111/s0203_re.hea',
'patient112/s0169_re.hea',
'patient113/s0018cre.hea',
'patient113/s0018lre.hea',
'patient114/s0012_re.hea',
'patient115/s0023_re.hea',
'patient116/s0302lre.hea',
'patient117/s0291lre.hea',
'patient117/s0292lre.hea',
'patient118/s0183_re.hea',
'patient119/s0001_re.hea',
'patient120/s0331lre.hea',
'patient121/s0311lre.hea',
'patient122/s0312lre.hea',
'patient123/s0224_re.hea',
'patient125/s0006_re.hea',
'patient126/s0154_re.hea',
'patient127/s0342lre.hea',
'patient127/s0383lre.hea',
'patient128/s0182_re.hea',
'patient129/s0189_re.hea',
'patient130/s0166_re.hea',
'patient131/s0273lre.hea',
'patient133/s0393lre.hea',
'patient135/s0334lre.hea',
'patient136/s0205_re.hea',
'patient137/s0392lre.hea',
'patient138/s0005_re.hea',
'patient139/s0223_re.hea',
'patient140/s0019_re.hea',
'patient141/s0307lre.hea',
'patient142/s0351lre.hea',
'patient143/s0333lre.hea',
'patient144/s0341lre.hea',
'patient145/s0201_re.hea',
'patient146/s0007_re.hea',
'patient147/s0211_re.hea',
'patient148/s0335lre.hea',
'patient149/s0202are.hea',
'patient149/s0202bre.hea',
'patient150/s0287lre.hea',
'patient151/s0206_re.hea',
'patient152/s0004_re.hea',
'patient153/s0391lre.hea',
'patient154/s0170_re.hea',
'patient155/s0301lre.hea',
'patient156/s0299lre.hea',
'patient157/s0338lre.hea',
'patient158/s0294lre.hea',
'patient158/s0295lre.hea',
'patient159/s0390lre.hea',
'patient160/s0222_re.hea',
'patient162/s0193_re.hea',
'patient163/s0034_re.hea',
'patient164/s0024are.hea',
'patient164/s0024bre.hea',
'patient165/s0322lre.hea',
'patient165/s0323lre.hea',
'patient166/s0275lre.hea',
'patient167/s0200_re.hea',
'patient168/s0032_re.hea',
'patient168/s0033_re.hea',
'patient169/s0328lre.hea',
'patient169/s0329lre.hea',
'patient170/s0274lre.hea',
'patient171/s0364lre.hea',
'patient172/s0304lre.hea',
'patient173/s0305lre.hea',
'patient174/s0300lre.hea',
'patient174/s0324lre.hea',
'patient174/s0325lre.hea',
'patient175/s0009_re.hea',
'patient176/s0188_re.hea',
'patient177/s0366lre.hea',
'patient178/s0011_re.hea',
'patient179/s0176_re.hea',
'patient180/s0374lre.hea',
'patient180/s0475_re.hea',
'patient180/s0476_re.hea',
'patient180/s0477_re.hea',
'patient180/s0490_re.hea',
'patient180/s0545_re.hea',
'patient180/s0561_re.hea',
'patient181/s0204are.hea',
'patient181/s0204bre.hea',
'patient182/s0308lre.hea',
'patient183/s0175_re.hea',
'patient184/s0363lre.hea',
'patient185/s0336lre.hea',
'patient186/s0293lre.hea',
'patient187/s0207_re.hea',
'patient188/s0365lre.hea',
'patient189/s0309lre.hea',
'patient190/s0040_re.hea',
'patient190/s0041_re.hea',
'patient191/s0340lre.hea',
'patient192/s0048_re.hea',
'patient193/s0008_re.hea',
'patient194/s0310lre.hea',
'patient195/s0337lre.hea',
'patient196/s0002_re.hea',
'patient197/s0350lre.hea',
'patient197/s0403lre.hea',
'patient198/s0402lre.hea',
'patient198/s0415lre.hea',
'patient199/s0404lre.hea',
'patient200/s0405lre.hea',
'patient201/s0420_re.hea',
'patient201/s0423_re.hea',
'patient202/s0421_re.hea',
'patient202/s0422_re.hea',
'patient203/s0424_re.hea',
'patient204/s0425_re.hea',
'patient205/s0426_re.hea',
'patient206/s0427_re.hea',
'patient207/s0428_re.hea',
'patient208/s0429_re.hea',
'patient208/s0430_re.hea',
'patient209/s0431_re.hea',
'patient210/s0432_re.hea',
'patient211/s0433_re.hea',
'patient212/s0434_re.hea',
'patient213/s0435_re.hea',
'patient214/s0436_re.hea',
'patient215/s0437_re.hea',
'patient216/s0438_re.hea',
'patient217/s0439_re.hea',
'patient218/s0440_re.hea',
'patient219/s0441_re.hea',
'patient220/s0442_re.hea',
'patient221/s0443_re.hea',
'patient222/s0444_re.hea',
'patient223/s0445_re.hea',
'patient223/s0446_re.hea',
'patient224/s0447_re.hea',
'patient225/s0448_re.hea',
'patient226/s0449_re.hea',
'patient227/s0450_re.hea',
'patient228/s0451_re.hea',
'patient229/s0452_re.hea',
'patient229/s0453_re.hea',
'patient230/s0454_re.hea',
'patient231/s0455_re.hea',
'patient232/s0456_re.hea',
'patient233/s0457_re.hea',
'patient233/s0458_re.hea',
'patient233/s0459_re.hea',
'patient233/s0482_re.hea',
'patient233/s0483_re.hea',
'patient234/s0460_re.hea',
'patient235/s0461_re.hea',
'patient236/s0462_re.hea',
'patient236/s0463_re.hea',
'patient236/s0464_re.hea',
'patient237/s0465_re.hea',
'patient238/s0466_re.hea',
'patient239/s0467_re.hea',
'patient240/s0468_re.hea',
'patient241/s0469_re.hea',
'patient241/s0470_re.hea',
'patient242/s0471_re.hea',
'patient243/s0472_re.hea',
'patient244/s0473_re.hea',
'patient245/s0474_re.hea',
'patient245/s0480_re.hea',
'patient246/s0478_re.hea',
'patient247/s0479_re.hea',
'patient248/s0481_re.hea',
'patient249/s0484_re.hea',
'patient250/s0485_re.hea',
'patient251/s0486_re.hea',
'patient251/s0503_re.hea',
'patient251/s0506_re.hea',
'patient252/s0487_re.hea',
'patient253/s0488_re.hea',
'patient254/s0489_re.hea',
'patient255/s0491_re.hea',
'patient256/s0492_re.hea',
'patient257/s0493_re.hea',
'patient258/s0494_re.hea',
'patient259/s0495_re.hea',
'patient260/s0496_re.hea',
'patient261/s0497_re.hea',
'patient262/s0498_re.hea',
'patient263/s0499_re.hea',
'patient264/s0500_re.hea',
'patient265/s0501_re.hea',
'patient266/s0502_re.hea',
'patient267/s0504_re.hea',
'patient268/s0505_re.hea',
'patient269/s0508_re.hea',
'patient270/s0507_re.hea',
'patient271/s0509_re.hea',
'patient272/s0510_re.hea',
'patient273/s0511_re.hea',
'patient274/s0512_re.hea',
'patient275/s0513_re.hea',
'patient276/s0526_re.hea',
'patient277/s0527_re.hea',
'patient278/s0528_re.hea',
'patient278/s0529_re.hea',
'patient278/s0530_re.hea',
'patient279/s0531_re.hea',
'patient279/s0532_re.hea',
'patient279/s0533_re.hea',
'patient279/s0534_re.hea',
'patient280/s0535_re.hea',
'patient281/s0537_re.hea',
'patient282/s0539_re.hea',
'patient283/s0542_re.hea',
'patient284/s0543_re.hea',
'patient284/s0551_re.hea',
'patient284/s0552_re.hea',
'patient285/s0544_re.hea',
'patient286/s0546_re.hea',
'patient287/s0547_re.hea',
'patient287/s0548_re.hea',
'patient288/s0549_re.hea',
'patient289/s0550_re.hea',
'patient290/s0553_re.hea',
'patient291/s0554_re.hea',
'patient292/s0555_re.hea',
'patient292/s0556_re.hea',
'patient293/s0557_re.hea',
'patient293/s0558_re.hea',
'patient294/s0559_re.hea'
]

# Combine header data into a single DataFrame
combined_headers = pd.DataFrame()

for hea_file in hea_files:
    df_header = read_header(f'/content/gdrive/MyDrive/Records/Database/{hea_file}')
    combined_headers = pd.concat([combined_headers, df_header], ignore_index=True)

directory = '/content/gdrive/MyDrive/Records'
# File path for saving the Excel file
file_path = os.path.join(directory, 'patients_details1.csv')

# Export to csv
combined_headers.to_csv(file_path, index=False)